###  MicroGrad demo

In [6]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mycrograd_debug.engine_debug import Value
from mycrograd_debug.nn_debug import Neuron, Layer, MLP
from mycrograd_debug.drawviz_debug import draw_dot,draw_nn
import pprint
np.random.seed(1337)
random.seed(1337)
number_of_samples=2
number_of_iterations=2
pp = pprint.PrettyPrinter(indent=4)

In [7]:
# make up a dataset

from sklearn.datasets import make_moons, make_blobs
X, y = make_moons(n_samples=number_of_samples, noise=0.1)
pp.pprint(X)
pp.pprint(y)
y = y*2 - 1 # make y be -1 or 1
pp.pprint(y)

array([[ 0.8572558 ,  0.03006228],
       [-0.05120403,  0.50473341]])
array([0, 1])
array([-1,  1])


In [8]:
# initialize a model 
# model = MLP(2, [16, 16, 1]) # 2-layer neural network
model = MLP(2, [2, 1], debug_bw=False) # 2-layer neural network
# print(model)
# pp.pprint(model.parameters())
Xb, yb = X, y
inputs = [list(map(Value, xrow)) for xrow in Xb]
pp.pprint(inputs)
for i in range(len(inputs)):
    for j in range(len(inputs[i])):
        inputs[i][j].type = "i"
pp.pprint(inputs)

Module nn MLP: structure [2, 2, 1]
[   [   Value(name=v280,layernumber=,neuronnumber=,weightnumber=,type=,data=0.8572557995669348, grad=0),
        Value(name=v281,layernumber=,neuronnumber=,weightnumber=,type=,data=0.03006227935826109, grad=0)],
    [   Value(name=v282,layernumber=,neuronnumber=,weightnumber=,type=,data=-0.051204029028939925, grad=0),
        Value(name=v283,layernumber=,neuronnumber=,weightnumber=,type=,data=0.5047334093296381, grad=0)]]
[   [   Value(name=v280,layernumber=,neuronnumber=,weightnumber=,type=i,data=0.8572557995669348, grad=0),
        Value(name=v281,layernumber=,neuronnumber=,weightnumber=,type=i,data=0.03006227935826109, grad=0)],
    [   Value(name=v282,layernumber=,neuronnumber=,weightnumber=,type=i,data=-0.051204029028939925, grad=0),
        Value(name=v283,layernumber=,neuronnumber=,weightnumber=,type=i,data=0.5047334093296381, grad=0)]]


In [9]:
# loss function
def loss():
    
    # forward the model to get scores
    scores = list(map(model, inputs))
    
    # svm "max-margin" loss
    losses = [(1 + -yi*scorei).relu() for yi, scorei in zip(yb, scores)]
    data_loss = sum(losses) * (1.0 / len(losses))
    # L2 regularization
    alpha = 1e-4
    reg_loss = alpha * sum((p*p for p in model.parameters()))
    total_loss = data_loss + reg_loss
    
    # also get accuracy
    accuracy = [(yi > 0) == (scorei.data > 0) for yi, scorei in zip(yb, scores)]
    return total_loss, sum(accuracy) / len(accuracy)

total_loss, acc = loss()
print(total_loss, acc)

Value(name=v348,layernumber=,neuronnumber=,weightnumber=,type=,data=0.9065883811424296, grad=0) 1.0


In [10]:


# optimization
for k in range(number_of_iterations):
    print("start")
    # pp.pprint(model.parameters())

    # forward
    total_loss, acc = loss()
    dot=draw_dot(total_loss)
    dot.render("images/opt_01_step%d" % k , format="jpg", view=True)


    # backward
    model.zero_grad()
    total_loss.backward()
    
    # update (sgd)
    learning_rate = 1.0 - 0.9*k/100
    for p in model.parameters():
        p.data -= learning_rate * p.grad
    
    if k % 1 == 0:
        print(f"step {k} loss {total_loss.data}, accuracy {acc*100}%")


start
step 0 loss 0.9065883811424296, accuracy 100.0%
start
step 1 loss 0.6796366507619458, accuracy 100.0%


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/tmeng12/snap/code/common/.cache/gio-modules/libgiolibproxy.so


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/tmeng12/snap/code/common/.cache/gio-modules/libgiolibproxy.so
